In [ ]:
!pip install smopy

In [1]:
from uo_bar_charts import BarChart
import datetime
chart = BarChart('people')

end_date = datetime.datetime.now()
last_monday = end_date - datetime.timedelta(days=end_date.weekday())
start_date = last_monday - datetime.timedelta(21)
chart.cache_data({'theme':'People'},{'theme':'People'},start_date,end_date,)


http://uoweb3.ncl.ac.uk:80/metadata/api/sensors/?theme=People
cacheing sensor:  PER_PEOPLE_BLACKETT-BOOTS(1/15)
cacheing sensor:  PER_PEOPLE_THE_CORE_LINE_0(2/15)
cacheing sensor:  PER_PEOPLE_THE_CORE_LINE_1(3/15)
cacheing sensor:  PER_PEOPLE_BLACKETT-NORTHUMBERLAND-W(4/15)
cacheing sensor:  PER_PEOPLE_USB_LINE_0(5/15)
cacheing sensor:  PER_PEOPLE_NORTHUMERLAND_LINE_SHORT_DISTANCE_HEAD_6(6/15)
cacheing sensor:  PER_PEOPLE_NORTHUMERLAND_LINE_SHORT_DISTANCE_HEAD_3(7/15)
cacheing sensor:  PER_PEOPLE_NORTHUMERLAND_LINE_LONG_DISTANCE_HEAD_1(8/15)
cacheing sensor:  PER_PEOPLE_NORTHUMERLAND_LINE_SHORT_DISTANCE_HEAD_5(9/15)
cacheing sensor:  PER_PEOPLE_NORTHUMERLAND_LINE_LONG_DISTANCE_HEAD_0(10/15)
cacheing sensor:  PER_PEOPLE_NORTHUMERLAND_LINE_SHORT_DISTANCE_HEAD_2(11/15)
cacheing sensor:  PER_PEOPLE_NORTHUMERLAND_LINE_SHORT_DISTANCE_HEAD_0(12/15)
cacheing sensor:  PER_PEOPLE_NORTHUMERLAND_LINE_SHORT_DISTANCE_HEAD_1(13/15)
cacheing sensor:  PER_PEOPLE_NORTHUMERLAND_LINE_SHORT_DISTANCE_HEAD_4